In [ ]:

import pandas as pd
import heapq
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Beispiel-Koordinatenliste mit Längen- und Breitengraden
coordinates = [
    [40.7128, -74.0060],  # New York
    [34.0522, -118.2437],  # Los Angeles
    [51.5074, -0.1278],    # London
    [48.8566, 2.3522],     # Paris
    [37.7749, -122.4194]   # San Francisco
]

# Anzahl der nächsten Nachbarn, die Sie suchen möchten
k = 1

# Erstellen des k-NN-Modells
knn_model = NearestNeighbors(n_neighbors=k)
knn_model.fit(coordinates)

# Wählen Sie eine Koordinate, für die Sie den nächsten Nachbarn finden möchten
query_coordinate = [38.8951, -77.0364]  # Washington, DC

# Finden Sie die nächsten Nachbarn der ausgewählten Koordinate
distances, indices = knn_model.kneighbors([query_coordinate])

# Zeigen Sie die Informationen über den nächsten Nachbarn an
nearest_neighbor_index = indices[0][0]
nearest_neighbor_distance = distances[0][0]
nearest_neighbor = coordinates[nearest_neighbor_index]

print("Die nächste Koordinate zu", query_coordinate, "ist", nearest_neighbor)
print("Entfernung:", nearest_neighbor_distance)

In [ ]:
import numpy as np
import folium
from sklearn.neighbors import BallTree
from IPython.display import display

# Beispiel-Ladestationen mit Längen- und Breitengraden
charging_stations = [
    {"name": "Station 1", "latitude": 52.5200, "longitude": 13.4050},
    {"name": "Station 2", "latitude": 48.8566, "longitude": 2.3522},
    # Weitere Ladestationen hier...
]

# Wählen Sie einen Punkt (z.B. [38.8951, -77.0364]) aus, für den Sie die nächstgelegene Ladestation finden möchten
query_point = [38.8951, -77.0364]

# Koordinaten der Ladestationen
station_coords = np.array([(station["latitude"], station["longitude"]) for station in charging_stations])

# Erstellen des BallTree-Index für die Ladestationen
tree = BallTree(station_coords, leaf_size=2)

# Finden der nächstgelegenen Ladestation mit k-NN
distances, indices = tree.query([query_point], k=1)

# Index der nächsten Ladestation finden
nearest_station_index = indices[0][0]

# Koordinaten der nächsten Ladestation
nearest_station_lat = charging_stations[nearest_station_index]["latitude"]
nearest_station_long = charging_stations[nearest_station_index]["longitude"]

# Karte von Deutschland mit Folium erstellen
map_center = [51.1657, 10.4515]  # Zentrum von Deutschland
m = folium.Map(location=map_center, zoom_start=6)

# Marker für Ladestationen hinzufügen
for station in charging_stations:
    popup_content = f"<b>Name:</b> {station['name']}"
    folium.Marker([station['latitude'], station['longitude']], popup=popup_content).add_to(m)

# Marker für die nächstgelegene Ladestation hinzufügen
popup_content = f"<b>Name:</b> {charging_stations[nearest_station_index]['name']}<br><b>Nächste Station</b>"
folium.Marker([nearest_station_lat, nearest_station_long], icon=folium.Icon(color='red'), popup=popup_content).add_to(m)

# Anzeigen der Karte in der Notebook-Zelle
display(m)


hier die Version mit Mapping

In [ ]:
import pandas as pd
import numpy as np
import folium
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic
from IPython.display import display

# Laden der CSV-Datei mit Pandas
data = pd.read_csv("/Users/nowasika/Documents/TechLabs/Team02_ChargeIT/datasets/2.landing/id_ladesaeulenregister.csv")  # Ersetzen Sie "your_data.csv" durch den tatsächlichen Dateinamen

# Koordinaten der Ladestationen aus den Spalten "Latitude" und "Longitude"
charging_stations = data[["Breitengrad", "Längengrad"]].values

# K-NN Algorithmus mit k=2 erstellen (um die nächstgelegenen 2 Stationen zu finden)
knn = NearestNeighbors(n_neighbors=2)
knn.fit(charging_stations)

# Finden der nächstgelegenen Ladestationen für alle Stationen
distances, nearest_station_indices = knn.kneighbors(charging_stations)

# Karte von Deutschland mit Folium erstellen
map_center = [51.1657, 10.4515]  # Zentrum von Deutschland
m = folium.Map(location=map_center, zoom_start=6)

# Marker für Ladestationen hinzufügen
for station_lat, station_long in charging_stations:
    folium.Marker([station_lat, station_long]).add_to(m)

# Marker für die nächstgelegenen Ladestationen hinzufügen
for i, nearest_indices in enumerate(nearest_station_indices):
    nearest_station_lat, nearest_station_long = charging_stations[nearest_indices[1]]
    distance_km = geodesic(charging_stations[i], (nearest_station_lat, nearest_station_long)).kilometers
    popup_content = f"Nächste Station<br>Entfernung: {distance_km:.2f} km"
    folium.Marker([nearest_station_lat, nearest_station_long], icon=folium.Icon(color='red'), popup=popup_content).add_to(m)

# Anzeigen der Karte in der Notebook-Zelle
display(m)


In [ ]:
## Version 3: Zu viele Knotenpunkte werden angezeigt.
import pandas as pd
import numpy as np
import folium
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic
from IPython.display import display

# Laden der CSV-Datei mit Pandas :: Immer den Lokalenpfad der Datei rein kopieren!!::
data = pd.read_csv("/Users/Takomon/Desktop/IT/techlabs/Team02_ChargeIT/datasets/2.landing/ladesauele_id.csv") 

# Koordinaten der Ladestationen aus den Spalten "Latitude" und "Longitude"
charging_stations = data[["Breitengrad", "Längengrad"]].values

# K-NN Algorithmus mit k=5 erstellen (um die nächstgelegenen 5 Stationen zu finden)
knn = NearestNeighbors(n_neighbors=5)
knn.fit(charging_stations)

# Eingabe von Startpunkt, Zielpunkt und Reichweite durch den Benutzer
start_lat = float(input("Geben Sie die Breitengrad des Startpunkts ein: "))
start_long = float(input("Geben Sie die Längengrad des Startpunkts ein: "))
ziel_lat = float(input("Geben Sie die Breitengrad des Zielpunkts ein: "))
ziel_long = float(input("Geben Sie die Längengrad des Zielpunkts ein: "))
reichweite_km = float(input("Geben Sie die Reichweite Ihres Fahrzeugs in km ein: "))

# Berechnen der Entfernung zwischen Start- und Zielpunkt
start_point = (start_lat, start_long)
ziel_point = (ziel_lat, ziel_long)
gesamtdistanz_km = geodesic(start_point, ziel_point).kilometers

# Finden der nächstgelegenen Ladestationen für Start- und Zielpunkt
distances_start, nearest_start_indices = knn.kneighbors([(start_lat, start_long)])
distances_ziel, nearest_ziel_indices = knn.kneighbors([(ziel_lat, ziel_long)])

# Karte von Deutschland mit Folium erstellen
map_center = [51.1657, 10.4515]  # Zentrum von Deutschland
m = folium.Map(location=map_center, zoom_start=6)

# Marker für Ladestationen hinzufügen
for station_lat, station_long in charging_stations:
    folium.Marker([station_lat, station_long]).add_to(m)

# Marker für die nächstgelegenen Ladestationen auf dem Weg hinzufügen
nearest_station_indices_start = nearest_start_indices[0]
nearest_station_indices_ziel = nearest_ziel_indices[0]

for idx in nearest_station_indices_start:
    if idx in nearest_station_indices_ziel:
        station_lat, station_long = charging_stations[idx]
        distance_km = geodesic(start_point, (station_lat, station_long)).kilometers
        popup_content = f"Nächste Station<br>Entfernung: {distance_km:.2f} km"
        folium.Marker([station_lat, station_long], icon=folium.Icon(color='green'), popup=popup_content).add_to(m)

# Anzeigen der Karte in der Notebook-Zelle
display(m)

# Ausgabe der Gesamtdistanz und Überprüfung, ob die Reichweite ausreicht
print(f"Gesamtdistanz zwischen Start- und Zielpunkt: {gesamtdistanz_km:.2f} km")
if gesamtdistanz_km <= reichweite_km:
    print("Ihre Reichweite reicht aus, um das Ziel zu erreichen.")
else:
    print("Ihre Reichweite reicht nicht aus, um das Ziel zu erreichen.")


In [3]:
data = pd.read_csv("/Users/takomon/Documents/IT/TechLabs/team2/team02/datasets/2.landing/2.ladesauele.csv") 
display(data)

,Betreiber,Straße,Hausnummer,Postleitzahl,Ort,Bundesland,Breitengrad,Längengrad,Nennleistung Ladeeinrichtung [kW],Art der Ladeeinrichung,Anzahl Ladepunkte,Steckertypen1,P1 [kW],Steckertypen2,P2 [kW],Steckertypen3,P3 [kW],Steckertypen4,P4 [kW]
0,Albwerk GmbH & Co. KG,Ennabeurer Weg,0,72535,Heroldstatt,Baden-Württemberg,48.442398,9.659075,22.0,Normalladeeinrichtung,2,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN
1,smopi®,Albstraße,14,72535,Heroldstatt,Baden-Württemberg,48.449353,9.672201,22.0,Normalladeeinrichtung,4,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0
2,Albwerk GmbH & Co. KG,Parkplatz Campingplatz,0,72589,Westerheim,Baden-Württemberg,48.510500,9.609000,22.0,Normalladeeinrichtung,2,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN
3,EnBW mobility+ AG und Co.KG,Hauptstraße,91c,73340,Amstetten,Baden-Württemberg,48.578534,9.874840,98.0,Schnellladeeinrichtung,2,AC Kupplung Typ 2,43.0,"DC Kupplung Combo, DC CHAdeMO",50.0,NaN,NaN,NaN,NaN
4,SWU Energie GmbH,Sterngasse,6,88481,Balzheim,Baden-Württemberg,48.179959,10.076977,22.0,Normalladeeinrichtung,1,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44846,Energieversorgung Apolda GmbH,Adolf-Aber-Straße,0,99510,Apolda,Thüringen,51.020983,11.510480,50.0,Schnellladeeinrichtung,1,"DC Kupplung Combo, DC CHAdeMO",50.0,NaN,NaN,NaN,NaN,NaN,NaN
44847,Energieversorgung Apolda GmbH,Martinskirchplatz,0,99510,Apolda,Thüringen,51.024391,11.511938,22.0,Normalladeeinrichtung,2,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN
44848,Energieversorgung Apolda GmbH,Schlossplatz,0,99510,Apolda,Thüringen,51.021359,11.514027,22.0,Normalladeeinrichtung,2,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN
44849,TEAG Mobil GmbH,Sulzaer Straße,0,99518,Großheringen,Thüringen,51.107188,11.659489,30.0,Normalladeeinrichtung,2,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN


In [7]:


import pandas as pd
import numpy as np
import folium
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic
from IPython.display import display

# Laden der CSV-Datei mit Pandas
data = pd.read_csv("/Users/takomon/Documents/IT/TechLabs/team2/team02/datasets/2.landing/2.ladesauele.csv") 

# Koordinaten der Ladestationen aus den Spalten "Latitude" und "Longitude"
charging_stations = data[["Breitengrad", "Längengrad"]].values

# K-NN Algorithmus mit k=5 erstellen (um die nächstgelegenen 5 Stationen zu finden)
knn = NearestNeighbors(n_neighbors=5)
knn.fit(charging_stations)

# Eingabe von Startpunkt, Zielpunkt und Reichweite durch den Benutzer
start_lat = '50.916401'#float(input("Geben Sie die Breitengrad des Startpunkts ein: "))
start_long = '6.934364'#float(input("Geben Sie die Längengrad des Startpunkts ein: "))
ziel_lat = '51.362496'#float(input("Geben Sie die Breitengrad des Zielpunkts ein: "))
ziel_long = '6.782188'#float(input("Geben Sie die Längengrad des Zielpunkts ein: "))
reichweite_km = '100'#float(input("Geben Sie die Reichweite Ihres Fahrzeugs in km ein: "))

#Beispiel Werte: Lat: 50.916401, Lon: 6.934364
#Beispiel Werte: Lat: 51.362496, Lon: 6.782188

# Berechnen der Entfernung zwischen Start- und Zielpunkt
start_point = (start_lat, start_long)
ziel_point = (ziel_lat, ziel_long)
gesamtdistanz_km = geodesic(start_point, ziel_point).kilometers

# Finden der nächstgelegenen Ladestationen für Start- und Zielpunkt
distances_start, nearest_start_indices = knn.kneighbors([(start_lat, start_long)])
distances_ziel, nearest_ziel_indices = knn.kneighbors([(ziel_lat, ziel_long)])

# Karte von Deutschland mit Folium erstellen
map_center = [51.1657, 10.4515]  # Zentrum von Deutschland
m = folium.Map(location=map_center, zoom_start=6)

# Marker für Start- und Ziel-Locations hinzufügen
folium.Marker([start_lat, start_long], icon=folium.Icon(color='blue'), popup='Startpunkt').add_to(m)
folium.Marker([ziel_lat, ziel_long], icon=folium.Icon(color='red'), popup='Zielpunkt').add_to(m)

# Linie für die Strecke zwischen Start- und Zielpunkt hinzufügen
folium.PolyLine([start_point, ziel_point], color="purple", weight=2.5, opacity=1).add_to(m)


# Marker für die nächstgelegenen Ladestationen auf dem Weg hinzufügen
nearest_station_indices_start = nearest_start_indices[0]
nearest_station_indices_ziel = nearest_ziel_indices[0]

# Alle Ladestationen auf dem Weg
stations_on_path = []

for idx in nearest_station_indices_start:
    if idx in nearest_station_indices_ziel:
        station_lat, station_long = charging_stations[idx]
        distance_km = geodesic(start_point, (station_lat, station_long)).kilometers
        stations_on_path.append((station_lat, station_long, distance_km))

# Sortieren Sie die Ladestationen nach Entfernung
stations_on_path.sort(key=lambda x: x[2])

# Begrenzen Sie die Anzahl der anzuzeigenden Ladestationen auf 5
stations_on_path = stations_on_path[:5]

# Marker für Ladestationen hinzufügen
for station_lat, station_long, distance_km in stations_on_path:
    popup_content = f"Nächste Station<br>Entfernung: {distance_km:.2f} km"
    folium.Marker([station_lat, station_long], icon=folium.Icon(color='green'), popup=popup_content).add_to(m)

# Anzeigen der Karte in der Notebook-Zelle
display(m)

# Ausgabe der Gesamtdistanz und Überprüfung, ob die Reichweite ausreicht
print(f"Gesamtdistanz zwischen Start- und Zielpunkt: {gesamtdistanz_km:.2f} km")
if gesamtdistanz_km <= reichweite_km:
    print("Ihre Reichweite reicht aus, um das Ziel zu erreichen.")
else:
    print("Ihre Reichweite reicht nicht aus, um das Ziel zu erreichen.")


ValueError: dtype='numeric' is not compatible with arrays of bytes/strings.Convert your data to numeric values explicitly instead.

In [ ]:
#URL für Koordinaten Umrechner: [For TESTING]

#https://www.koordinaten-umrechner.de/decimal/50.916401,6.934364?karte=OpenStreetMap&zoom=9

#https://www.koordinaten-umrechner.de/decimal/51.362496,6.782188?karte=OpenStreetMap&zoom=8

In [2]:
import osmnx as ox
import pandas as pd
import numpy as np
import folium
from geopy.distance import geodesic
from IPython.display import display

# Benutzerdefinierte Funktion zum Laden eines Straßennetzwerks um den Startpunkt
def load_graph_around_point(point, radius_m, network_type='all'):
    # Verwenden Sie OSMnx, um das Straßennetzwerk im Radius um den Punkt zu laden
    graph = ox.graph_from_point(point, dist=radius_m, network_type=network_type)
    return graph

# Eingabe von Startpunkt, Zielpunkt und Reichweite durch den Benutzer
start_lat = '50.916401'#float(input("Geben Sie die Breitengrad des Startpunkts ein: "))
start_long = '6.934364'#float(input("Geben Sie die Längengrad des Startpunkts ein: "))
ziel_lat = '51.362496'#float(input("Geben Sie die Breitengrad des Zielpunkts ein: "))
ziel_long = '6.782188'#float(input("Geben Sie die Längengrad des Zielpunkts ein: "))
reichweite_km = '100'#float(input("Geben Sie die Reichweite Ihres Fahrzeugs in km ein: "))

# Berechnen der Entfernung zwischen Start- und Zielpunkt
start_point = (start_lat, start_long)
ziel_point = (ziel_lat, ziel_long)
gesamtdistanz_km = geodesic(start_point, ziel_point).kilometers

# Umrechnen der Reichweite in Meter
reichweite_meter = reichweite_km * 1000

# Laden des Straßennetzwerks von OpenStreetMap um den Startpunkt
graph = load_graph_around_point((start_lat, start_long), radius_m=reichweite_meter * 1.1, network_type='all')  # Reichweite leicht erweitern

# Konvertieren des Straßennetzwerks in einen gerichteten Graphen
G = ox.add_edge_bearings(ox.project_graph(graph))

# Finden der nächsten Straßenknoten für Start- und Zielpunkt
start_node = ox.distance.nearest_nodes(G, start_long, start_lat)
ziel_node = ox.distance.nearest_nodes(G, ziel_long, ziel_lat)

# Prüfen, ob die Straßenroute innerhalb der Reichweite liegt
if not G.has_edge(start_node, ziel_node):
    print("Die gewünschte Route liegt außerhalb Ihrer Reichweite.")
else:
    # Berechnen der Straßenroute zwischen Start- und Zielpunkt
    route = ox.shortest_path(G, start_node, ziel_node, weight='length')

    # Extrahieren der Koordinaten der Routenpunkte
    route_coordinates = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]

    # Karte von Deutschland mit Folium erstellen
    map_center = [51.1657, 10.4515]  # Zentrum von Deutschland
    m = folium.Map(location=map_center, zoom_start=6)

    # Linie für die echte Straßenroute hinzufügen
    folium.PolyLine(route_coordinates, color="purple", weight=2.5, opacity=1).add_to(m)

    # Marker für Start- und Zielpunkt hinzufügen
    folium.Marker([start_lat, start_long], icon=folium.Icon(color='blue'), popup='Startpunkt').add_to(m)
    folium.Marker([ziel_lat, ziel_long], icon=folium.Icon(color='red'), popup='Zielpunkt').add_to(m)

    # Anzeigen der Karte in der Notebook-Zelle
    display(m)

    # Ausgabe der Gesamtdistanz und Überprüfung, ob die Reichweite ausreicht
    print(f"Gesamtdistanz zwischen Start- und Zielpunkt: {gesamtdistanz_km:.2f} km")
    if gesamtdistanz_km <= reichweite_km:
        print("Ihre Reichweite reicht aus, um das Ziel zu erreichen.")
    else:
        print("Ihre Reichweite reicht nicht aus, um das Ziel zu erreichen.")



TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
import requests

# Zugriff auf die OpenStreetMap-API-Token
secrets = openai_secret_manager.get_secret("openstreetmap")

# Eingabe von Startpunkt, Zielpunkt und Reichweite durch den Benutzer
start_lat = float(input("Geben Sie die Breitengrad des Startpunkts ein: "))
start_long = float(input("Geben Sie die Längengrad des Startpunkts ein: "))
ziel_lat = float(input("Geben Sie die Breitengrad des Zielpunkts ein: "))
ziel_long = float(input("Geben Sie die Längengrad des Zielpunkts ein: "))
reichweite_km = float(input("Geben Sie die Reichweite Ihres Fahrzeugs in km ein: "))

# Berechnen der Entfernung zwischen Start- und Zielpunkt
start_point = (start_lat, start_long)
ziel_point = (ziel_lat, ziel_long)

# Verwenden Sie die OpenStreetMap-API, um die Route zwischen Start- und Zielpunkt zu erhalten
api_url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={secrets['api_key']}&start={start_long},{start_lat}&end={ziel_long},{ziel_lat}"
response = requests.get(api_url)

if response.status_code == 200:
    route_data = response.json()
    # Die erhaltenen Daten enthalten die Koordinaten der Route
    route_coordinates = [(point["coordinates"][1], point["coordinates"][0]) for point in route_data["features"][0]["geometry"]["coordinates"]]
    
    # Jetzt können Sie diese Koordinaten auf einer Karte darstellen und Ihre Ladestationen hinzufügen
    # Fügen Sie Ihre Kartenvisualisierung hier ein
else:
    print("Fehler beim Abrufen der Routendaten.")
